**Download and parse interest rates for deposits**

Was used for my bachelor's thesis

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def get_soup(page_int):
    r = requests.get(f'https://www.banki.ru/products/deposits/?page={page_int}')
    r_text = r.content.decode('utf-8')
    soup = BeautifulSoup(r_text)
    return soup

In [ ]:
# name/class of element might change
# so we find some image that stays the same
# and go to the top using .parent

def get_anchor(page_int=1, image_name='Газпромбанк'):
    soup = get_soup(page_int)
    anchor = soup.find('img', {'alt': image_name}).parent.parent.parent.parent.parent.parent
    anchor = anchor['class']
    return anchor

In [ ]:
def parse_element(element):
    bank = element.find('img')['alt']
    divs = element.find_all('div')
    # names of elements might change
    # so we use order
    name = divs[2].text
    rate = divs[5].text
    term = divs[6].text
    money = divs[8].text
    return [i.strip('\n') for i in (bank, name, rate, term, money)]

In [ ]:
def parse_one_page(page_int, anchor):
    soup = get_soup(page_int)
    lists = [parse_element(element) for element in soup.find_all('div', class_=anchor[0])]
    df = pd.DataFrame(lists, columns=['Банк', 'Вклад', 'Ставка', 'Срок', 'Сумма'])
    return df

In [ ]:
def parse_all_pages():
    anchor = get_anchor()
    df_list = []
    i = 1
    while True:
        df = parse_one_page(i, anchor)
        # no more pages
        if len(df) == 0:
            break
        df_list.append(df)
        i += 1
    return pd.concat(df_list)

In [ ]:
df = parse_all_pages()

In [ ]:
df.head()

In [ ]:
def parse_text(var_name, regex_range, regex_value):
    extract_range = df[var_name].str.extract(regex_range)
    extract_value = df[var_name].str.extract(regex_value)
    regex_df = extract_range.combine_first(extract_value)
    regex_df.columns = [f'Мин. {var_name}', f'Макс. {var_name}']
    return regex_df

In [ ]:
# parse_text('Ставка', 
#            r'([\d,]+) — ([\d,]+)%', 
#            r'([\d,]+)%')

# parse_text('Срок', 
#            r'([\d,]+) — ([\d,]+) дн.', 
#            r'([\d,]+) дн.')

# parse_text('Сумма', 
#            r'([\d\s]+) — ([\d\s]+) ₽', 
#            r'от ([\d\s]+) ₽')

In [ ]:
pattern_range = df['Ставка'].str.extract(r'([\d,]+) — ([\d,]+)%')
pattern_value = df['Ставка'].str.extract(r'([\d,]+)%')
regex_res = pattern_1.combine_first(pattern_2)
regex_res.columns = ['Мин. ставка', 'Макс. ставка']
for i in regex_res.columns:
    df[i] = c[i]

In [ ]:
pattern_1 = df['Срок'].str.extract(r'([\d,]+) — ([\d,]+) дн.')
pattern_2 = df['Срок'].str.extract(r'([\d,]+) дн.')
regex_res = pattern_1.combine_first(pattern_2)
regex_res.columns = ['Мин. срок', 'Макс. срок']
for i in c.columns:
    df[i] = c[i]

In [ ]:
pattern_1 = df['Сумма'].str.extract(r'([\d\s]+) — ([\d\s]+) ₽')
pattern_2 = df['Сумма'].str.extract(r'от ([\d\s]+) ₽')
regex_res = pattern_1.combine_first(pattern_2)
regex_res.columns = ['Мин. сумма', 'Макс. сумма']
for i in regex_res.columns:
    df[i] = regex_res[i]

In [ ]:
df

In [ ]:
# df.to_csv('bankiru_rates.csv')